# Requêter Wikidata grâce à SPARQLWrapper

#### Imports nécéssaires
SPARQLWrapper est une bibliothèque Python qui facilite l'interaction avec des endpoints SPARQL pour interroger des bases de données RDF. Elle sert d’interface entre Python et des bases de connaissances comme Wikidata, DBpedia ou d'autres triplestores compatibles avec SPARQL.

In [2]:
import time
import random
import json

import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

#### Ouvrir le JSON et vérifier rapidement son contenu

In [4]:
with open("enriched_openlibrary.json", "r", encoding="utf-8") as f:
    data = json.load(f)

data[3:6]

[{'key': '/works/OL145191W',
  'title': 'Picasso',
  'authors': [{'key': '/authors/OL44790A', 'name': 'Pablo Picasso'},
   {'key': '/authors/OL3225101A', 'name': 'Jean-Louis Andral'},
   {'key': '/authors/OL139551A', 'name': 'Pierre Daix'}],
  'first_publish_year': 1926,
  'wikidataSubjectID': ['Q42934', 'Q38166'],
  'wikidataArtistID': []},
 {'key': '/works/OL495761W',
  'title': 'Evangeline',
  'authors': [{'key': '/authors/OL33009A',
    'name': 'Henry Wadsworth Longfellow'}],
  'first_publish_year': 1847,
  'wikidataSubjectID': [],
  'wikidataArtistID': []},
 {'key': '/works/OL14877882W',
  'title': 'Salomé',
  'authors': [{'key': '/authors/OL20646A', 'name': 'Oscar Wilde'}],
  'first_publish_year': 1893,
  'wikidataSubjectID': [],
  'wikidataArtistID': []}]

#### Extraire les IDs OpenLibrary des auteurs

In [25]:
all_author_ids = list(set(
    author["key"].split("/")[-1]  # Extraire l'ID après "/authors/"
    for work in data for author in work.get("authors", []) if work.get("wikidataSubjectID") or work.get("wikidataArtistID")
))

print(f"{len(all_author_ids)} auteurs trouvés")


12392 auteurs trouvés


#### Regroupement des ID par chunks pour faciliter le requêtage de Wikidata

In [26]:
chunks = [all_author_ids[x:x+100] for x in range(0, len(all_author_ids), 100)]
print(len(chunks))

124


#### Configuration du point d'accès SPARQL

In [27]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

#### Définition de la fonction de requêtage
La fonction `get_wikidata_info_batch(values)` interroge Wikidata pour récupérer des informations sur une liste d’auteurs identifiés par leur identifiant Open Library (P648). Elle exécute une requête SPARQL pour extraire l'ID Wikidata, le genre, la date de naissance, la date de décès (si disponible) et la nationalité des auteurs, puis formate ces données dans un dictionnaire où chaque clé correspond à un identifiant Open Library.

In [28]:
def get_wikidata_info_batch(values):
    """ Interroge Wikidata pour récupérer les infos d'une liste d'auteurs """
    query = f"""
    SELECT ?artist ?artistLabel ?birth ?death ?gender ?genderLabel ?country ?countryLabel ?ol_id
    WHERE {{
      VALUES ?ol_id {{ { values } }}
      ?artist wdt:P648 ?ol_id;
              wdt:P21 ?gender;
              wdt:P569 ?birth.
      OPTIONAL {{ ?artist wdt:P570 ?death. }}
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    results = sparql.query().convert()
    
    author_info = {}
    for item in results['results']['bindings']:
        ol_id = item["ol_id"]["value"]
        author_info[ol_id] = {
            "wikidataID": item["artist"]["value"].split(sep="/")[-1],
            "gender": item["genderLabel"]["value"],
            "genderId": item["gender"]["value"].split(sep="/")[-1],
            "birthdate": item["birth"]["value"],
            "deathdate": item.get("death", {}).get("value", ""), 
            "countrycitizenship": {
                entry["country"]["value"].split(sep="/")[-1]: entry["countryLabel"]["value"]
                for entry in results['results']['bindings'] if "country" in entry
            }
        }
    return author_info

#### Requêtage
Ce code récupère des informations sur des auteurs en batch à partir de Wikidata et les stocke dans un fichier JSON. Il parcourt des listes d'ID (chunks), construit une requête pour chaque groupe d'ID, interroge Wikidata via get_wikidata_info_batch(), ajoute les résultats à author_data, puis attend un temps aléatoire (entre 6 et 12 secondes) avant de continuer pour éviter de surcharger l'API. Enfin, les données sont enregistrées dans ol_authors_info.json avec un format lisible (indent=4).

In [29]:
author_data = []
sleep_options = [6, 8, 10, 12]
option = random.choice

for id_list in chunks:
    values = ' '.join(f'"{id}"' for id in id_list)
    author_info = get_wikidata_info_batch(values)
    author_data.append(author_info)
    option = random.choice(sleep_options)
    time.sleep(option)

with open("ol_authors_info.json", "w") as f:
    json.dump(author_data, f, indent=4)


#### Enregistrement et comptage

In [32]:
with open('ol_authors_info.json', 'r') as f:
    data = json.load(f)

count = []
for chunk in data: 
    count.append((len(chunk)))

print(sum(count))

916
